# Project 2
## Eniacs product strategy

Your analyse can finally settle an ongoing debate for the mgmt: whether or not it’s beneficial to discount products.

*Marketing opinion*\
Marketing Team Lead is convinced that offering discounts is beneficial in the long run:
1. discounts improve customer acquisition, satisfaction and retention, and allows the company to grow.

*Investors opinion*\
Main investors in the Board are worried about offering aggressive discounts:
1. recent quarterly results showed an increase in number of orders, but a decrease in the total revenue
2. focus in quality segment, rather than offer lowest prices in market

*Your analytical and business skills are needed to give clarity in the following aspects:*
- How products should be classified into different categories in order to simplify reports and analysis.
- What is the distribution of product prices across different categories.
- How many products are being discounted.
- How big are the offered discounts as a percentage of the product prices.
- How seasonality and special dates (Christmas, Black Friday) affect sales.
- How could data collection be improved.

`Present to company board your analysis (short and concise, but also engaging and convincing)`

In [ ]:
# Preparation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
path = 'eniac/clean/'
orderlines = pd.read_csv(path + 'orderlines_cl.csv')
orders = pd.read_csv(path + 'orders_cl.csv')
brands = pd.read_csv(path + 'brands_cl.csv')
products = pd.read_csv(path + 'products_cl.csv')

In [ ]:
orders['created_date'] = pd.to_datetime(orders['created_date'])
orderlines['date'] = pd.to_datetime(orderlines['date'])

## Part from Rias

### Ratio of quality products (+500 euro) on total monthly sales 17/18

In [722]:
qlty = (
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed"')
    .assign(price_cat = lambda x: pd.cut(x['total_price'], [0, 500, 8500]),
            y_m = lambda x: x['date'].dt.to_period('M'))
    .groupby(['y_m','price_cat'])
    .agg(count = ('id_order','count'))
    .reset_index()
    .assign(total_count = lambda x: x.groupby('y_m')['count'].transform('sum'))
    .assign(ratio = lambda x: 100 * x['count'] / x['total_count'],
            price_cat = lambda x: x['price_cat'].astype('str'))    
    .query('price_cat == "(500, 8500]"')
)
qlty

In [ ]:
qlty.to_excel('eniac/categories.xlsx')

### How many products are being discounted?

In [579]:
# How many products have been sold overall?
t = (orderlines.merge(orders, how='left', left_on='id_order', right_on='order_id').query('state == "Completed"').shape) 

# How much of them had a discount more then 0 euros?
d = (
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed"')
    .merge(products, how='left', left_on='sku', right_on='sku')
    .assign(disc_perc = lambda x: round((1 - x['unit_price'] / x['price'])*100), 
            disc = lambda x: x['price'] - x['unit_price'],)
    .query('disc > 0')
    .shape
)
print(d[0]*100/t[0])

90.49965380094359


### Buildung categories

In [ ]:
## how many sku's exist?
orderlines.value_counts('sku')#.head(20)#.to_excel('eniac/top_skus.xlsx')#, 'id_order')#['id_order']#.head(50)
## 7951 sku's overall

In [700]:
# build categories
prep_cat = (
orderlines
    .merge(products, how='left')
    .merge(orders, left_on='id_order', right_on='order_id', how='left')
    .query('state == "Completed"')
    .assign(short = lambda x: x['name'].str[:45])
    .groupby(['sku', 'short', 'desc'])
    .agg({'product_quantity':'sum'})
    .reset_index()
    .sort_values('product_quantity', ascending=False)
)

regexp_dict = {
    'licences':'(AppleCare|Apple Care|Care|license|License)',
    'accessories':'(Kit|Ram|Keyboard|Car charger|Case|case|funda|Housing|housing|casing|folder|protect|cover|sleeve|Screensaver|shell|cable|connector|Lightning to USB|Wall socket|power strip|battery|Mouse|Pencil|USB-C|adapter)',
    'refurbished':'(refurbished|reconditioned|like new)',
    'wearables':'(^.{0,6}apple watch|smartwatch|smart watch|headset|headphones)',
    'storage':'(SSD|NAS|HDD)',
    'phone':('Apple iPhone|Phone'),
    'computers':'(iMac|MacBook|Mac Pro|iPad|Tablet|PC)'
}

temp = prep_cat.copy().assign(category = 'unknown')

import numpy as np

for val in regexp_dict.items(): 
    label = val[0]
    regexp = re.compile(val[1], flags=re.IGNORECASE)
    temp = (
    temp
        .assign(
            category = lambda x: np.where(
                (x['desc'].str.contains(regexp, regex=True)) &
                (x['category'] == 'unknown'), label, x['category'])))

temp['category'].value_counts()

accessories    3295
computers       805
phone           615
unknown         552
storage         344
refurbished     208
wearables       145
licences         17
Name: category, dtype: int64

In [691]:
temp.to_excel('eniac/categories.xlsx')